In [8]:
# MOUNT G_DRIVE
from pathlib import Path
from IPython import get_ipython
on_colab = 'google.colab' in str(get_ipython())

if on_colab:
  from google.colab import drive
  drive.mount("/content/gdrive")

PATH =  "/content/gdrive/My Drive/DeepLearning/MODELS/" if on_colab else "./"
PATH_DATASET = "/content/gdrive/My Drive/DeepLearning/DATASETS/" if on_colab else "./"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


MODELS

In [9]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Softmax, Conv2d, Dropout
import torch.nn as nn
from torch.nn import LSTM, GRU, Linear, Softmax
from torchtext.vocab import GloVe
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
from tqdm import tqdm
from torchtext.datasets import IMDB, AG_NEWS, YahooAnswers
from torchtext.data import to_map_style_dataset
from torch.utils.data import Dataset
import pandas as pd
from torchtext.data.utils import get_tokenizer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss


class BidirectionalLSTMClassifier(torch.nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.LSTM = LSTM(50, hidden_size, num_layers=num_layers,
                         batch_first=True, bidirectional=True)
        self.linear = Linear(2 * hidden_size, num_classes)
        self.softmax = Softmax(dim=1)

    def forward(self, x):
        _, (h_n, _) = self.LSTM(x)
        h_forward = h_n[2 * self.num_layers - 2]
        h_backward = h_n[2 * self.num_layers - 1]
        y = self.linear(torch.cat((h_forward, h_backward), 1))
        return self.softmax(y)


class CNNClassifier(torch.nn.Module):
    def __init__(self, num_classes, in_channels, out_channels, kernel_heights, pad=0, stri=1, embed_dim=50, drop=0.2):
        super().__init__()
        self.conv1 = Conv2d(in_channels, out_channels[0], kernel_size=(kernel_heights[0], embed_dim), stride=stri, padding=pad)
        self.conv2 = Conv2d(in_channels, out_channels[1], kernel_size=(kernel_heights[1], embed_dim), stride=stri, padding=pad)
        self.conv3 = Conv2d(in_channels, out_channels[2], kernel_size=(kernel_heights[2], embed_dim), stride=stri, padding=pad)
        self.drop = Dropout(drop)
        self.fc = Linear(sum(out_channels), num_classes)
        self.soft = Softmax(dim=1)

    def _conv_n_maxpool_1d(self, input, conv_layer):

        conved = conv_layer(input) # conved.size() = (batch_size, out_channels[0], dim, 1)
        reld = F.relu(conved.squeeze(3)) # reld.size() = (batch_size, out_channels[0], dim)
        max_out = F.max_pool1d(reld, reld.size()[2]).squeeze(2) # maxpool_out.size() = (batch_size, out_channels[0])

        return max_out

    def forward(self, x):
        # x.size() = (batch_size, num_seq, embed_dim)
        x = x.unsqueeze(1) # x.size() = (batch_size, 1, num_seq, embed_dim)

        out_1 = self._conv_n_maxpool_1d(x, self.conv1)
        out_2 = self._conv_n_maxpool_1d(x, self.conv2)
        out_3 = self._conv_n_maxpool_1d(x, self.conv3)

        cat_out = torch.cat((out_1, out_2, out_3), dim=1)

        drop = self.drop(cat_out)
        fc_out = self.fc(drop)
        out = self.soft(fc_out)

        return out

def get_child(model, *arg):
    res = model
    for i in arg:
        res = list(res.children())[i]
    return res

def freeze_model(model):
    for param in model.parameters():
            param.requires_grad = False
            
def unfreeze_model(model):
    for param in model.parameters():
            param.requires_grad = True

def count_parameters(model, trainable_only = True):
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())

def custom_freezer(model):
    unfreeze_model(model)

    ## freeze whole BertLayer
    for c in model.children():
        if str(c).startswith('Bert'):
            freeze_model(c)
            
    ## unfreeze top 2 layer in BertEncoder
    bert_encoder = get_child(model, 0, 1, 0)
    for i in range(1, 3):
        m = bert_encoder[-i] 
        unfreeze_model(m)
        
    ## unfreeze Pooling layer
    bert_pooling = get_child(model, 0, 2)
    unfreeze_model(bert_pooling)

    print('Trainable parameters: {}'.format(count_parameters(model, True)))
    return model

class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()

        self.bert = BertForSequenceClassification.from_pretrained(
            'bert-base-uncased', num_labels=num_classes)
        self.bert = custom_freezer(self.bert)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, token_type_ids, attention_mask):
        y = self.bert(input_ids, token_type_ids, attention_mask)
        return self.softmax(y.logits)

In [10]:
class AdversarialClassificationDataset(Dataset):
    def __init__(self, dataset_path, num_classes, tokenizer, model):
        self.num_classes = num_classes
        self.dataset = pd.read_csv(dataset_path)
        self.tokenizer = tokenizer
        self.model = model

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        label = self.dataset["label"][idx]
        text = self.dataset["txt"][idx]
        
        if self.model == 'BERT':
            return label, self.tokenizer(text, padding="max_length", return_tensors='pt', max_length=512, truncation=True)
        else:
            return label, self.tokenizer(text)

class ClassificationDataset(Dataset):
    def __init__(self, dataset, num_classes, tokenizer, model):
        self.num_classes = num_classes
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.model = model

    def __len__(self):
        return self.dataset.__len__()

    def __getitem__(self, idx):
        label, text = self.dataset.__getitem__(idx)
        if type(label) == str:
            if label == 'neg':
                label = 0
            else:
                label = 1
        else:
            label = int(label) - 1

        if self.model == 'BERT':
            return label, self.tokenizer(text, padding="max_length", return_tensors='pt', max_length=512, truncation=True)
        else:
            return label, self.tokenizer(text)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#embedding = GloVe(name='6B', dim=50)
#tokenizer = get_tokenizer("basic_english")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

'''def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _tokens) in batch:
        label_list.append(_label)
        embed = embedding.get_vecs_by_tokens(_tokens)
        text_list.append(embed)
    text_list = pad_sequence(text_list, batch_first=True)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    return label_list.to(device), text_list.to(device)'''

def collate_BERT(batch):
    label_list, input_ids, token_type_ids, attention_mask = [], [], [], []
    for (_label, _dic) in batch:
        label_list.append(_label)
        input_ids.append(_dic['input_ids'])
        token_type_ids.append(_dic['token_type_ids'])
        attention_mask.append(_dic['attention_mask'])
    label_list = torch.tensor(label_list, dtype=torch.int64).to(device)
    input_ids = torch.cat(input_ids, dim=0).to(device)
    token_type_ids = torch.cat(token_type_ids, dim=0).to(device)
    attention_mask = torch.cat(attention_mask, dim=0).to(device)
    return label_list, input_ids, token_type_ids, attention_mask

In [11]:
# SELECT DATASET & MODEL

DATASET_NAME = "AG_NEWS_ADV.csv"
MODEL = "BERT"
DATASET = "AG_NEWS"
num_classes = 4
BATCH_SIZE = 64
NUM_EPOCHS = 5
SHUFFLE = True
CLUSTER = False

train_set = AdversarialClassificationDataset(PATH_DATASET+DATASET_NAME, num_classes, tokenizer, MODEL)

test_set = AG_NEWS(split="test")
test_set = to_map_style_dataset(test_set)
test_set = ClassificationDataset(test_set, num_classes, tokenizer, MODEL)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_BERT, shuffle=SHUFFLE)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, collate_fn=collate_BERT, shuffle=SHUFFLE)

1.86MB [00:00, 19.3MB/s]                  


In [12]:
def evaluate(model, data_loader, loss=CrossEntropyLoss()):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        if MODEL == "BERT":
            for idx, (labels, input_ids, token_type_ids, attention_mask) in enumerate(data_loader):
                predicted_label = model(
                    input_ids, token_type_ids, attention_mask)
                loss_ = loss(predicted_label, labels)
                total_acc += (predicted_label.argmax(1) == labels).sum().item()
                total_count += labels.size(0)
        else:
            for idx, (labels, text) in enumerate(data_loader):
                predicted_label = model(text)
                loss_ = loss(predicted_label, labels)
                total_acc += (predicted_label.argmax(1) == labels).sum().item()
                total_count += labels.size(0)

    return total_acc / total_count


def train(model, optimizer, train_loader, loss=CrossEntropyLoss(), log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    pbar = tqdm(total=len(train_loader),
                desc=f'Epoch [{epoch + 1}/{NUM_EPOCHS}]')

    if MODEL == 'BERT':
        for idx, (labels, input_ids, token_type_ids, attention_mask) in enumerate(train_loader):
            output = model(input_ids, token_type_ids, attention_mask)
            loss_ = loss(output, labels)
            optimizer.zero_grad()
            loss_.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_acc += (output.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
            if not CLUSTER:
                pbar.update()
            if idx % log_interval == 0 and idx > 0:
                if not CLUSTER:
                    pbar.set_postfix(
                        loss=loss_, accuracy=total_acc / total_count)
                total_acc, total_count = 0, 0

        pbar.close()
    else:
        for idx, (labels, text) in enumerate(train_loader):
            output = model(text)
            loss_ = loss(output, labels)
            optimizer.zero_grad()
            loss_.backward()
            optimizer.step()
            total_acc += (output.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
            if not CLUSTER:
                pbar.update()
            if idx % log_interval == 0 and idx > 0:
                if not CLUSTER:
                    pbar.set_postfix(
                        loss=loss_, accuracy=total_acc / total_count)
                total_acc, total_count = 0, 0

        pbar.close()


In [13]:
model = BERTClassifier(num_classes).to(device)
#optim = Adam(model.parameters())
NUM_EPOCHS = 3
optim = AdamW(model.parameters(), lr=3e-5, correct_bias=False)
for epoch in range(NUM_EPOCHS):
    train(model, optim, train_loader)
    torch.save({'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict()}, PATH + MODEL + '_' + DATASET + '_' + 'PWWS' + '.pt')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Trainable parameters: 14769412


Epoch [1/3]:   1%|          | 22/1931 [02:08<3:06:39,  5.87s/it]

KeyboardInterrupt: ignored

In [ ]:
test_accuracy = evaluate(model, test_loader)
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.9010526315789473
